In [212]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
#load data and set index
universe = pd.read_csv('universe.csv')
df = pd.DataFrame(universe)
df.set_index('ticker', inplace=True)
#fix some columns
df.roa = df.roa.str.replace('%','').astype('float')
df.roe = df.roe.str.replace('%','').astype('float')
df.short_to_float = df.short_to_float.str.replace('%','').astype('float')
df.oper_margin = df.oper_margin.str.replace('%','').astype('float')
df.profit_margin = df.profit_margin.str.replace('%','').astype('float')
df.loc['TSLA','trail_pe'] = 1640
df.trail_pe = df.trail_pe.astype('float')
df.loc['NLOK','px2book'] = 1190
df.px2book = df.px2book.astype('float')
df['ticker2'] = df.index
df['analyst_est'] = ((df['1y_est'] / df['px'])-1)*100
df.perf_1y = df.perf_1y.str.replace('%','').astype('float')

In [ ]:
df

In [ ]:
#companies with good roe and roa
q=.85
roa = df.loc[df.roa>df.roa.quantile(q),['roa']]
roe = df.loc[df.roe>df.roe.quantile(q),['roe']]

good_ro = pd.merge(roa,roe, how='inner', left_index=True, right_index=True)
good_ro.describe()

In [ ]:
#companies with good operating figures
q=.85
curr_ratio = df.loc[df.curr_ratio>df.curr_ratio.quantile(q),['curr_ratio']]
oper_margin = df.loc[df.oper_margin>df.oper_margin.quantile(q),['oper_margin']]
profit_margin = df.loc[df.profit_margin>df.profit_margin.quantile(q), ['profit_margin']]

merge = pd.merge(curr_ratio,oper_margin, how='inner', left_index=True, right_index=True)
good_ops = pd.merge(merge,profit_margin, how='inner', left_index=True, right_index=True)
good_ops.describe()

In [205]:
#add ons
add_ons = df[['div_yld','_5y_div_yld','analyst_est','name']]
add_ons2 = df[['div_yld','_5y_div_yld','analyst_est','name','pe']]

In [ ]:
#companies with reasonable px valuations
q=.38
peg = df.loc[df.peg<df.peg.quantile(q),['peg']]
px2sales = df.loc[df.px2sales<df.px2sales.quantile(q),['px2sales']]
px2book = df.loc[df.px2book<df.px2book.quantile(q),['px2book']]

merge1 = pd.merge(px2book,peg, how='inner', left_index=True, right_index=True)
good_px = pd.merge(merge1,px2sales, how='inner', left_index=True, right_index=True)
good_px.describe()

In [ ]:
#companies with reasonable PEs
q=.22
pe = df.loc[df.pe<df.pe.quantile(q),['pe']]
trail_pe = df.loc[df.trail_pe<df.trail_pe.quantile(q),['trail_pe']]
fwd_pe = df.loc[df.fwd_pe<df.fwd_pe.quantile(q),['fwd_pe']]

merge = pd.merge(pe,trail_pe, how='inner', left_index=True, right_index=True)
good_pe = pd.merge(merge,fwd_pe, how='inner', left_index=True, right_index=True)
good_pe.describe()

In [ ]:
#companies with good ev ratios
q=.25
ev2rev = df.loc[df.ev2rev<df.px2book.quantile(q), ['ev2rev']]
ev2ebitda = df.loc[df.ev2ebitda<df.ev2ebitda.quantile(q),['ev2ebitda']]

good_ev = pd.merge(ev2rev, ev2ebitda, how='inner', left_index=True, right_index=True)
good_ev.describe()

In [206]:
# good_ops_ro
good_ops_ro = pd.merge(good_ops, good_ro, how='inner', left_index=True, right_index=True)
pd.merge(good_ops_ro, add_ons2, how='inner', left_index=True, right_index=True)

,curr_ratio,oper_margin,profit_margin,roa,roe,div_yld,_5y_div_yld,analyst_est,name,pe
ticker,,,,,,,,,,
INTU,3.09,30.31,25.10,18.68,44.31,0.58,0.85,2.299300,Intuit Inc.,54.90
MXIM,5.67,33.54,30.66,13.15,39.53,NaN,2.98,-5.877396,"Maxim Integrated Products, Inc.",35.40
TXN,4.28,40.97,38.69,19.82,61.84,2.34,2.45,0.223189,Texas Instruments Incorporated,30.03
VRTX,5.02,46.41,43.70,17.94,36.71,NaN,NaN,34.866264,Vertex Pharmaceuticals Incorporated,20.46
ZTS,2.82,36.23,25.42,11.85,52.89,0.61,0.60,12.503041,Zoetis Inc.,47.39


In [190]:
good_pe_px = pd.merge(good_pe, good_px, how='inner', left_index=True, right_index=True)
good_pe_px_ev = pd.merge(good_pe_px, good_ev, how='inner', left_index=True, right_index=True)
good_pe_px_ev = pd.merge(good_pe_px_ev, add_ons, how='inner', left_index=True, right_index=True)
good_pe_px_ev

,pe,trail_pe,fwd_pe,px2book,peg,px2sales,ev2rev,ev2ebitda,div_yld,_5y_div_yld,analyst_est,name
ticker,,,,,,,,,,,,
CVS,12.17,11.93,9.54,1.37,1.26,0.36,0.64,9.30,2.72,2.73,16.059783,CVS Health Corporation
DHI,11.28,11.07,9.59,2.38,0.87,1.36,1.42,9.00,0.96,1.13,14.403835,"D.R. Horton, Inc."
DISH,12.91,11.80,11.17,1.20,1.18,1.18,1.88,9.41,NaN,NaN,27.548122,DISH Network Corporation
LEN,12.05,11.59,9.41,1.58,0.76,1.25,1.43,9.96,1.06,0.39,0.962352,Lennar Corporation
TSN,11.15,11.15,10.98,1.54,1.23,0.55,0.79,7.66,2.72,1.67,16.666667,"Tyson Foods, Inc."


In [169]:
q=.15
good_esg = df.loc[df.esg_score<df.esg_score.quantile(q),['esg_score']]

In [170]:
q=.40
board = df.loc[df.board<df.board.quantile(q),['board']]
sh_holdr_rgts = df.loc[df.sh_holdr_rgts<df.sh_holdr_rgts.quantile(q),['sh_holdr_rgts']]
comp = df.loc[df.comp<df.comp.quantile(q),['comp']]

merge = pd.merge(sh_holdr_rgts,board, how='inner', left_index=True, right_index=True)
good_gov = pd.merge(merge,comp, how='inner', left_index=True, right_index=True)
# good_gov

In [207]:
gov = pd.merge(good_gov,esg, how='inner', left_index=True, right_index=True)
pd.merge(gov,add_ons2, how='inner', left_index=True, right_index=True)


,sh_holdr_rgts,board,comp,esg_score,div_yld,_5y_div_yld,analyst_est,name,pe
ticker,,,,,,,,,
ACN,4.0,3.0,4.0,11.0,1.37,1.67,8.645789,Accenture plc,31.88
BBY,1.0,2.0,3.0,12.0,1.85,2.46,1.965047,"Best Buy Co., Inc.",18.22
CBRE,3.0,1.0,3.0,9.0,NaN,NaN,-7.803838,"CBRE Group, Inc.",22.07
MCO,3.0,3.0,4.0,12.0,0.80,1.07,14.117563,Moody's Corporation,29.01
REG,1.0,3.0,1.0,12.0,4.64,3.67,-0.487805,Regency Centers Corporation,185.02


In [203]:
#most bullish analyst estimates:
bull = df['analyst_est'].sort_values(ascending=False).head(20)
add_ons = df[['name','sector','div_yld','pe','beta']]
pd.merge(bull, add_ons, how='left',left_index=True, right_index=True)\
.sort_values(by='analyst_est', ascending=False)

,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,
FOX,67.479675,Fox Corporation,Communication Services,1.51,12.17,NaN
VRTX,34.866264,Vertex Pharmaceuticals Incorporated,Healthcare,NaN,20.46,0.64
CNC,34.633984,Centene Corporation,Healthcare,NaN,19.22,0.51
REGN,33.024830,"Regeneron Pharmaceuticals, Inc.",Healthcare,NaN,16.07,0.16
NEM,30.267215,Newmont Corporation,Basic Materials,2.68,18.35,0.25
MRK,28.632593,"Merck & Co., Inc.",Healthcare,3.48,26.88,0.44
FDX,27.940714,FedEx Corporation,Industrials,1.01,27.47,1.31
DISH,27.548122,DISH Network Corporation,Communication Services,NaN,12.91,2.00
FISV,25.405605,"Fiserv, Inc.",Technology,NaN,77.49,0.91


In [204]:
#most bearish analyst estimates:
bear = df['analyst_est'].sort_values(ascending=True).head(20)
add_ons = df[['name','sector','div_yld','pe','beta']]
pd.merge(bear, add_ons, how='left',left_index=True, right_index=True)\
.sort_values(by='analyst_est', ascending=True)

,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,
VIAC,-36.153846,ViacomCBS Inc.,Communication Services,1.68,NaN,1.81
AAL,-29.723367,American Airlines Group Inc.,Industrials,NaN,NaN,1.83
UA,-27.121291,"Under Armour, Inc.",Consumer Cyclical,NaN,NaN,1.19
ILMN,-25.880945,"Illumina, Inc.",Healthcare,NaN,104.59,1.02
DISCA,-24.917673,"Discovery, Inc.",Communication Services,NaN,25.24,1.51
TSLA,-24.803489,"Tesla, Inc.",Consumer Cyclical,NaN,1268.22,2.09
DISCK,-24.794872,"Discovery, Inc.",Communication Services,NaN,21.61,1.51
XRX,-24.512299,Xerox Holdings Corporation,Technology,4.24,27.68,1.89
LUMN,-16.000000,"Lumen Technologies, Inc.",Communication Services,8.01,NaN,NaN


In [233]:
q=.80
bear = df['analyst_est'].sort_values(ascending=True).head(25)
short_ratio = df.loc[df.short_ratio>df.short_ratio.quantile(q),['short_ratio']]
short_to_float = df.loc[df.short_to_float>df.short_to_float.quantile(q),['short_to_float']]
add_ons = df[['name','sector','div_yld','pe','beta']]

short = pd.merge(short_ratio,short_to_float, how='inner', left_index=True, right_index=True)
short_bear = pd.merge(short,bear, how='inner', left_index=True, right_index=True)
short_bear = pd.merge(short_bear,add_ons, how='inner', left_index=True, right_index=True)
short_bear

,short_ratio,short_to_float,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,,,
DISCA,4.10,33.35,-24.917673,"Discovery, Inc.",Communication Services,NaN,25.24,1.51
EXPE,3.86,10.18,-10.319525,"Expedia Group, Inc.",Consumer Cyclical,NaN,NaN,1.72
IRM,6.63,16.28,-9.356197,Iron Mountain Incorporated,Real Estate,7.44,72.26,0.84
LUMN,3.86,7.51,-16.000000,"Lumen Technologies, Inc.",Communication Services,8.01,NaN,NaN
LYV,5.54,9.48,-15.078973,"Live Nation Entertainment, Inc.",Communication Services,NaN,NaN,1.38
STX,3.71,10.76,-9.812310,Seagate Technology plc,Technology,3.70,18.93,1.16
VIAC,5.09,16.25,-36.153846,ViacomCBS Inc.,Communication Services,1.68,NaN,1.81
